In [ ]:
import os
import shutil
import time
import netCDF4 as nc

# test en local: 

In [ ]:
file_path = "D:/GCxGC_MS/DATA/624848_0017VE_J0.cdf"

In [ ]:
ds = nc.Dataset(file_path)

In [ ]:
t0 = time.time()
mv = ds["mass_values"][:]
iv = ds["intensity_values"][:]
print("Time to load variables:", time.time() - t0, "s")

# test sur docker

In [ ]:
file_path = "/app/data/624848_0017VE_J0.cdf"

In [ ]:
ds = nc.Dataset(file_path)

In [ ]:
t0 = time.time()
mv = ds["mass_values"][:]
iv = ds["intensity_values"][:]
print("Time to load variables:", time.time() - t0, "s")

# test copie sur docker avant d analyser

In [ ]:
source_path = file_path = "D:/GCxGC_MS/DATA/624848_0017VE_J0.cdf"
tmp_path = "/app/data/tmp"  # tmp a creer ?

In [ ]:
if not os.path.exists(tmp_path):
    print(f"Copying to fast access: {tmp_path}")
    t0 = time.time()
    shutil.copy(source_path, tmp_path)
    print("Time to copy:", time.time() - t0, "s")
else:
    print(f"Using cached copy: {tmp_path}")

In [ ]:
from netCDF4 import Dataset

t0 = time.time()
ds = Dataset(tmp_path)
mass_values = ds["mass_values"][:]
intensity_values = ds["intensity_values"][:]
print("Time to load variables:", time.time() - t0, "s")